In [1]:
from dotenv import load_dotenv
from portia import Config, Portia, PortiaToolRegistry, open_source_tool_registry, InMemoryToolRegistry
from portia.cli import CLIExecutionHooks

load_dotenv(override=True)

c:\Users\joshu\anaconda3\envs\RA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
from portia import Portia
from portia.config import default_config
from portia.open_source_tools.registry import example_tool_registry
from portia.clarification import MultipleChoiceClarification
from portia.plan_run import PlanRunState
from portia import LLMModel

my_config = Config.from_default()


my_config.models['planning_default_model_name'] = LLMModel.GPT_4_O

from my_custom_tools.PDFReaderTool import PDFReaderTool
example_tool_registry.register_tool(PDFReaderTool())
# Load example and custom tool registries into a single one
# Instantiate a Portia instance. Load it with the default config and with the tools above

portia = Portia(config=my_config,
                tools=PortiaToolRegistry(my_config)+example_tool_registry)

# Execute the plan from the user query
plan_run = portia.run('Run the PDFReaderTool and summarize the content of the PDF files returned by the tool.',)

# Check if the plan run was paused due to raised clarifications
while plan_run.state == PlanRunState.NEED_CLARIFICATION:
    # If clarifications are needed, resolve them before resuming the plan run
    for clarification in plan_run.get_outstanding_clarifications():
        # For each clarification, prompt the user for input
        print(f"{clarification.user_guidance}")
        user_input = input("Please enter a value:\n" 
                        + (("\n".join(clarification.options) + "\n") if "options" in clarification else ""))
        # Resolve the clarification with the user input
        plan_run = portia.resolve_clarification(clarification, user_input, plan_run)

    # Once clarifications are resolved, resume the plan run
    plan_run = portia.resume(plan_run)

# Serialise into JSON and print the output
print(plan_run.model_dump_json(indent=2))

2025-04-11 19:54:27.643 | INFO | portia.portia:plan:197 - Running planning_agent for query - Run the PDFReaderTool on the file "ddim.pdf" and output a summary of the content
2025-04-11 19:54:31.759 | INFO | portia.portia:plan:222 - Plan created with 2 steps | {'plan': 'plan-d83ca0ae-6d94-4437-ac45-571e71905357'}
2025-04-11 19:54:36.074 | INFO | portia.portia:_execute_plan_run:528 - Plan Run State is updated to PlanRunState.IN_PROGRESS. View in your Portia AI dashboard: https://app.portialabs.ai/dashboard/plan-runs?plan_run_id=prun-1d3a0c92-ad7c-49cb-a1dd-984d872ffea3
2025-04-11 19:54:36.076 | INFO | portia.portia:_execute_plan_run:551 - Executing step 0: Read the PDF file "ddim.pdf" from the local 'papers' folder. | {'plan': 'plan-d83ca0ae-6d94-4437-ac45-571e71905357', 'plan_run': 'prun-1d3a0c92-ad7c-49cb-a1dd-984d872ffea3'}
2025-04-11 19:54:38.478 | INFO | portia.tool_wrapper:run:115 - Invoking PDF reader tool with args: {'filename': 'ddim.pdf'}
2025-04-11 19:54:39.951 | ERROR | porti

In [32]:
outline = """
This demo requires you to have a folder with papers to summarize and find the subtopics of.
The folder should be named "papers" and should be in the same directory as this script.
"""

print(outline)



This demo requires you to have a folder with papers to summarize and find the subtopics of.
The folder should be named "papers" and should be in the same directory as this script.



In [33]:

task = (
    lambda: f"""
Please help me accomplish the following tasks:
Tasks:
- Read the papers in the folder 'papers' and summarize them in a single paragraph.
- Break down the summary into sub-topics that are needed to understand the papers
- Explain the sub-topics in detail, including any relevant background information.
"""
)

print("\nA plan will now be generated. Please wait...")


A plan will now be generated. Please wait...


In [ ]:
# Instantiate a Portia runner. Load it with the default config and with Portia cloud tools above.
# Use the CLIExecutionHooks to allow the user to handle any clarifications at the CLI.
my_config = Config.from_default()

from portia import LLMModel
my_config.models['planning_default_model_name'] = LLMModel.GPT_4_O

portia = Portia(
    config=my_config,
    tools=PortiaToolRegistry(my_config)+open_source_tool_registry,
    execution_hooks=CLIExecutionHooks(),
)


TypeError: PortiaToolRegistry.get_tool() missing 1 required positional argument: 'tool_id'

In [31]:
plan = portia.plan(task())
print("\nHere are the steps in the generated plan:")
[print(step.model_dump_json(indent=2)) for step in plan.steps]

# Iterate on the plan with the user until they are happy with it
ready_to_proceed = False
while not ready_to_proceed:
    user_input = input("Are you happy with the plan? (y/n):\n")
    if user_input == "y":
        ready_to_proceed = True
    else:
        user_input = input("Any additional guidance for the planner?:\n")
        constraints.append(user_input)
        plan = portia.plan(task())
        print("\nHere are the updated steps in the plan:")
        [print(step.model_dump_json(indent=2)) for step in plan.steps]

2025-04-11 19:09:20.371 | INFO | portia.portia:plan:197 - Running planning_agent for query - 
Please help me accomplish the following tasks, ensuring you take into account the following constraints: 
Tasks:
- Get my availability from Google Calendar tomorrow between 10:00 and 17:00
- If I am available, schedule a 30 minute meeting with joshua.waha@cogital.tech at a time that works for me with the title "Portia AI Demo" and a description of the meeting as "Test demo".
- and send an email to joshua.waha@cogital.tech with the details of the meeting you scheduled.

2025-04-11 19:09:26.697 | INFO | portia.portia:plan:222 - Plan created with 3 steps | {'plan': 'plan-15001112-ffef-42ad-8eb8-95a19ac55fe4'}

Here are the steps in the generated plan:
{
  "task": "Get the availability of the authenticated user from Google Calendar tomorrow between 10:00 and 17:00",
  "inputs": [],
  "tool_id": "portia:google:gcalendar:check_availability",
  "output": "$availability",
  "condition": null
}
{
  "ta

KeyboardInterrupt: 